In [1]:
import mysql.connector
import cv2
import os
import numpy as np

## NEGATIVE FACES - VGGFACE2 DATASET

In [2]:
# Connect to MySQL
database = mysql.connector.connect(
    user='root', password='jarvis@admin', host='192.168.1.97', port='3306')
cursor = database.cursor()


In [3]:
cursor.execute('USE VGG_Face2')
images_loc = '/coding-drive/DATASETS/negative-faces-raw/VGG-Face2/data/test/'
save_loc = '/coding-drive/DATASETS/face-recognition-data/test/Unknown/'

if os.path.exists(save_loc):
    print('Directory already exists')
else:
    os.mkdir(save_loc)
    print('Creating directory')

Directory already exists


In [4]:
count = 0
total_folders = len(os.listdir(images_loc))
# List all folders in images_loc
for folder in os.listdir(images_loc):
    # List all images in each folder
    for img_path in os.listdir(images_loc + folder):
        # check if image is already saved
        if os.path.exists(f'{save_loc}{folder}-{img_path}'):
            continue
        # Read image
        img = cv2.imread(os.path.join(images_loc, folder, img_path))
        # Get face coordinates
        cursor.execute(
            'SELECT X, Y, W, H FROM VGG_Face2_TEST WHERE PATH = %s', (os.path.join(folder, img_path.split('.')[0]),))
        X, Y, W, H = cursor.fetchone()
        # Crop face
        face = img[Y:Y+H, X:X+W]
        try:
            # Save face
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face = cv2.resize(face, (224, 224))
            cv2.imwrite(f'{save_loc}{folder}-{img_path}', face)
            # print('Saved ' + os.path.join(folder, img_path))
        except:
            # print('Error saving ' + os.path.join(folder, img_path), end='\r')
            pass
    count += 1
    print(f'Completed {count}/{total_folders} folders', end='\r')

## POSITIVE FACES

In [ ]:
person_name = input("Enter person name: ")

In [ ]:
images_loc = f'/coding-drive/DATASETS/raw-pos-face-data/devasheesh_test/'
save_loc = f'/coding-drive/DATASETS/face-recognition-data/test/{person_name}/'

if os.path.exists(save_loc):
    print('Directory already exists')
else:
    os.mkdir(save_loc)
    print('Creating directory')

In [ ]:
# using the Haar Cascade Classifier
def extract_frontal_face_harr(image_ndarray, grayscale=True, size=(150, 150)):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(
        '../haarcascade_frontalface_default.xml')
    # Convert into grayscale
    if grayscale:
        image_gray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image_ndarray
    # Detect faces
    faces_cord = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    # Return the face or None if not found
    if len(faces_cord) == 0:
        return None
    # Extract the face
    (x, y, w, h) = faces_cord[0]
    # Resize the image to 150x150
    image_gray_resized = cv2.resize(image_gray[y:y+w, x:x+h], size)
    # Return only the face part of the image
    return image_gray_resized


In [ ]:
count=0

for img_path in os.listdir(images_loc):
    img = cv2.imread(os.path.join(images_loc, img_path))
    try:
        face = extract_frontal_face_harr(image_ndarray=img, grayscale=True, size=(224, 224))
        if face is None:
            continue
        
        cv2.imwrite(f'{save_loc}{person_name}_{count}.jpg', face)
        # print('Saved ' + os.path.join(images_loc, img_path))
        count += 1
        print('Saved {} images'.format(count), end='\r')

    except Exception as e:
        print('Error saving ' + os.path.join(images_loc, img_path))
        print(e)
       